## Histone deacetylase 1 - part 2 (MMP identification)

### Import libraries

In [1]:
import pandas as pd

In [2]:
from tqdm import tqdm
tqdm.pandas()

import sys
sys.path.append('/home/daniel/wizepair2')
from mmpa.mmp import MMP

### Read in the dataset

In [3]:
df = pd.read_csv('hdac1_inhibitors_stripped.csv')
len(df.index)

108

### Create cartesian product of unique molecules tested in the same assay

In [4]:
df_pairs = pd.merge(df, df, on='assay_chembl_id')
df_pairs = df_pairs[['stripped_smiles_x', 'stripped_smiles_y']].drop_duplicates()
df_pairs

,stripped_smiles_x,stripped_smiles_y
0,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...
1,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
2,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...,N#Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
3,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...,Cc1cc(CN2CCC2)cnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
4,Cc1cc(CN2CCN(C(C)C)CC2)cnc1-c1ccc(C(=O)Nc2cccc...,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C#...
...,...,...
116,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...
117,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(Cl...
118,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C)...
119,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,Nc1ccccc1NC(=O)c1ccc(-c2ncc(CN3CCC3)cc2F)cc1


### Identify all pairs

In [5]:
df_pairs = df_pairs.progress_apply(lambda x: MMP(x.stripped_smiles_x, x.stripped_smiles_y, fuzziness=5).execute(), axis=1)

100%|██████████| 121/121 [04:12<00:00,  2.08s/it]


In [6]:
df_pairs = pd.json_normalize(df_pairs.explode())
df_pairs.sample(3).transpose()

,239,271,72
smiles1,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,CC(C)N1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c...,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1
smiles2,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(C#...,N#Cc1cccnc1-c1ccc(C(=O)Nc2ccccc2N)cc1,CCN1CCN(Cc2cnc(-c3ccc(C(=O)Nc4ccccc4N)cc3)c(Cl...
percentmcs,0.909091,0.676471,0.65625
radius,1,1,4
valid,False,True,True
smirks,[#6:14](-[#6:12](-[#6](-[H])(-[H])-[H])(-[#7:1...,[#6](-[#6](-[#6](-[H])(-[H])-[H])(-[#7]1-[#6](...,[#7]#[#6:17]-[#6:16]1:[#6:15](:[#6:9](:[#6:14]...
fragment1,NaN,[cH2][c]([cH2])[CH2][N]1[CH2][CH2][N]([CH]([CH...,[cH2][c]([cH2])-[c]1[n][cH][cH][cH][c]1[C]#[N]
fragment2,NaN,[cH2][cH][cH2],[cH2][c]([cH2])-[c]1[n][cH][c]([CH2][N]2[CH2][...


### Drop failures and write output to file

In [7]:
df_pairs = df_pairs[df_pairs.valid]
df_pairs.to_csv('hdac1_inhibitors_pairs.csv', index=False)